### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_kidney_disease/GSE66494'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
if "gene expression profiles" in "!Series_summary":
    is_gene_available = True
else:
    is_gene_available = False

# Check for the availability of variables in the Sample Characteristics Dictionary
sample_characteristics = {
    0: ['study set: discovery set', 'study set: validation set'],
    1: ['sample type: Renal biopsy specimens', 'sample type: Normal kidney total RNA'],
    2: ['specimen id: #01', 'specimen id: #02', 'specimen id: #03', 'specimen id: #04', 'specimen id: #05', 'specimen id: #06', 'specimen id: #07', 'specimen id: #08', 'specimen id: #09', 'specimen id: #10', 'specimen id: #11', 'specimen id: #12', 'specimen id: #13', 'specimen id: #14', 'specimen id: #15', 'specimen id: #16', 'specimen id: #17', 'specimen id: #18', 'specimen id: #19', 'specimen id: #20', 'specimen id: #21', 'specimen id: #22', 'specimen id: #23', 'specimen id: #24', 'specimen id: #26', 'specimen id: #27', 'specimen id: #28', 'specimen id: #29', 'specimen id: #30', 'specimen id: #31'],
    3: ['tissue: kidney', 'disease status: normal kidney'],
    4: ['disease status: chronic kidney disease (CKD)', float('nan')],
}

# Search for keys
for key, values in sample_characteristics.items():
    if any('chronic kidney disease' in str(v).lower() or 'ckd' in str(v).lower() for v in values):
        trait_row = key
    if any('age' in str(v).lower() for v in values):
        age_row = key
    if any('gender' in str(v).lower() or 'sex' in str(v).lower() for v in values):
        gender_row = key

# Define conversion functions
def convert_trait(value):
    try:
        val = value.split(':')[1].strip().lower()
        if val == 'chronic kidney disease (ckd)':
            return 1
        elif val == 'normal kidney':
            return 0
    except (IndexError, ValueError):
        return None

def convert_age(value):
    try:
        if isinstance(value, (float, int)):
            return value
        return float(value.split(':')[1].strip())
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    try:
        val = value.split(':')[1].strip().lower()
        if val == 'male':
            return 1
        elif val == 'female':
            return 0
    except (IndexError, ValueError):
        return None

# Save Metadata
save_cohort_info('GSE66494', './preprocessed/Chronic_kidney_disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_kidney_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Chronic_kidney_disease/trait_data/GSE66494.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
